In [130]:
import pandas as pd
from datetime import datetime
import sys
import re

In [5]:
pos1 = pd.read_csv('/home/lucia/phd_work/shareTask/CLpsych/data_sample_clpsych19/user10146.posts.csv')
pos2 = pd.read_csv('/home/lucia/phd_work/shareTask/CLpsych/data_sample_clpsych19/user10276.posts.csv')
neg1 = pd.read_csv('/home/lucia/phd_work/shareTask/CLpsych/data_sample_clpsych19/user-29175.posts.csv')
neg2 = pd.read_csv('/home/lucia/phd_work/shareTask/CLpsych/data_sample_clpsych19/user-44823.posts.csv')

In [29]:
file = pd.concat([pos1, pos2], ignore_index= True)
file2 = pd.concat([neg1, neg2], ignore_index= True)
file = pd.concat([file, file2], ignore_index= True) 
#convert timestamp to time
file['timestamp'] = file['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
file[1:10]

,post_id,user_id,timestamp,subreddit,post_title,post_body
1,275dra,10146,2014-06-02 22:05:47,OpTicGaming,[Throwback: BO2 MLG Anaheim] WHO'S THE BEST IN...,NaN
2,2760t9,10146,2014-06-03 02:12:28,ForeverAloneDating,20 [M4F] WA - Looking for the player 2 to my p...,"What's going on everybody? My name is Waf3l, b..."
3,277upp,10146,2014-06-03 17:33:23,CoDCompetitive,Who is your favorite Pro Team?,"I know we have the flair to back it up, but I ..."
4,27cmpe,10146,2014-06-05 02:53:13,r4r,"20 [M4F] Seattle - Can you be my ""It Girl""?","What's going on everybody? My name is Waf3l, b..."
5,27fi22,10146,2014-06-06 00:29:34,CodAW,"This may have been done to death, but *my thou...",*This will be a long post. I apologize in adva...
6,27ps40,10146,2014-06-09 18:15:45,OpTicGaming,Did anybody else chuckle when _PERSON_ yelled ...,NaN
7,28fb94,10146,2014-06-18 02:11:22,r4r,[Meta] I messed up.,"Hey r4r, I met a girl on here and I messed up..."
8,28lfmo,10146,2014-06-19 23:12:42,AskReddit,"What is the best ""bang for your buck"" option f...",NaN
9,28oave,10146,2014-06-20 21:26:19,GamerPals,20/M/PST - Looking for people to pub stomp wit...,Just looking for people to play with in my dow...


we build features on user level, for each user, we have Number of annotation unit, number of posts in sucideWatch, average length of post in suicideWatch, frequency of posting in all subreddit, frequency of posting in suicideWatch, words mentioned mental illness, words mention a physical condition, words mention medication, sentiment in posts, sentment in posts mentioned mental illness, number of posts without content

Now we create a dictionary that stores the posting pattern 

In [91]:
def PostingFreq(file):
    timeFreq = {} #dictionary that shows the average time a user post sth 
    preUser = None #previous user 
    preTime = None #posting time of the previous day
    dayVec = [] #a vector that shows the time difference between a previous day and the day after
    for userid, time in zip(file['user_id'], file['timestamp']):
        if preTime is None:
            freq = time 
        elif userid == preUser:
            #freq = previous day - day after
            freq = datetime.strptime(preTime, "%Y-%m-%d %H:%M:%S") - datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
            dayVec.append(freq.days)
            timeFreq[userid] = dayVec
        elif type(freq) is not str: # freq is not a datetime object if there is one case only
            dayVec = [freq.days] 

        preTime = time 
        preUser = userid
    return timeFreq

postingFrequency = PostingFreq(file)

In [92]:
for item in postingFrequency:
    print(postingFrequency[item])

[-4, -1, -1, -2, -1, -4, -9, -2, -1, -1, -2, -1, -1, -5, -2, -33, -1, -3, -26, -18, -7, -8, -4, -6, -29, -4, -1, -7, -71, -57, -3, -1, -5, -90, -4, -13, -10]
[-10, -1, -1, -5, -2, -8, -6, -4, -1, -1, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -5, -3, -6, -7, -6, -2, -5, -1, -1, -1, -2, -6, -3, -5, -1, -4, -37, -3, -2, -2, -1, -6, -10, -3, -2, -1, -1, -2, -3, -3, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -5, -11, -5, -2, -20, -1, -20, -24, -35, -15, -9, -13, -24, -71, -2, -1, -2, -1, -1, -8, -9, -3, -3, -7, -4, -10, -12, -7, -1, -1, -1, -1, -1, -1, -1, -9, -13, -2, -10, -5, -2, -1, -1, -14, -4, -1, -28, -3, -3, -5, -22, -2, -12, -21, -1, -14, -44, -4, -5, -10, -2, -3, -3, -5, -49, -26, -3, -10, -8, -1, -3, -12, -11, -3, -71, -115, -2, -2, -13, -12, -149]
[-149, -3, -2, -1, -27, -613, -31, -68, -123, -1, -20, -1, -9, -3, -6, -3, -1, -23]
[-23, -1, -2, -1, -25, -15, -23, -11, -7, -7, -64, -108, -113, -18, -37, -5, -2, -1, -21, -23, -1, -13, -1, -

we can get suicideWatch subreddit only and see the frequency of these posts 

In [101]:
suicideWatch = file[file['subreddit'].str.contains("SuicideWatch")]
suicideWatch

,post_id,user_id,timestamp,subreddit,post_title,post_body
14,2985q5,10146,2014-06-27 08:15:28,SuicideWatch,I'm done.,I'm just done with life. I'm nothing but a scr...
39,pxpri,10276,2012-02-20 12:36:25,SuicideWatch,Feeling suicidal,"I have aspergers, OCD, depression and all that..."
45,quvza,10276,2012-03-13 18:38:21,SuicideWatch,just swallowed a bunch of pills,"total of around 10 including panadol , augment..."


In [100]:
suicideWatchFrequency = PostingFreq(suicideWatch)
suicideWatchFrequency #here shows participant 10276's posting frequency on suicideWatch

{10276: [-23]}

now we can get number of posts and length of post

In [109]:
PostCount = suicideWatch.groupby(['user_id']).size().reset_index(name = 'counts')
PostCount

,user_id,counts
0,10146,1
1,10276,2


In [115]:
def stringCount(text):
    count = 0
    for item in text.split():
        count = count + 1
    return count 

suicideWatch.loc['wordCount'] = suicideWatch['post_body'].apply(lambda x: stringCount(x))
suicideWatch

/home/lucia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,post_id,user_id,timestamp,subreddit,post_title,post_body,wordCount
14,2985q5,10146.0,2014-06-27 08:15:28,SuicideWatch,I'm done.,I'm just done with life. I'm nothing but a scr...,80.0
39,pxpri,10276.0,2012-02-20 12:36:25,SuicideWatch,Feeling suicidal,"I have aspergers, OCD, depression and all that...",122.0
45,quvza,10276.0,2012-03-13 18:38:21,SuicideWatch,just swallowed a bunch of pills,"total of around 10 including panadol , augment...",22.0
wordCount,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def findText(text, wordList):
    for item in wordList:
        if item in text:
            return True
    return False


In [128]:
from tika import parser

raw = parser.from_file('/home/lucia/phd_work/shareTask/CLpsych/MYH-Glossary-of-terms.pdf')
print(raw['content'])













































 

Mental health & mental illness 

Glossary of common 
terminology 

www.mindyourheadyork.org 



 

2 

 
Glossary of common terminology 

in mental health & mental illness 
The definitions in this document come from a variety of sources including mental health charities Mind 

and Rethink, the National Health Service (NHS), Mayo Clinic, and other informational online resources. 

Mind Your Head does not comprise of health care professionals and as such we do not aim to invent 

new definitions where they exist, nor to challenge existing definitions. As a mental health awareness 

campaign we wish to compile an accessible resource that can be used by individuals to better under-

stand the terminology used in the literature or other resources in order to improve their knowledge of 

mental health and mental illness. 

We further hope that this glossary helps reduce misconceptions about mental health terminology. 

If you come across additio

In [132]:
phrase= "Yes! it worked"

patterns= ['[A-Z]+']

    
for p in patterns:
    match= re.findall(p, phrase)
    print(match)

['Y']
